<a href="https://colab.research.google.com/github/UrukHan/NeuroNetStockQuotes/blob/master/AlphaVintageApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [434]:
!pip install alpha_vantage

In [435]:
import requests
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from datetime import timedelta
from google.colab import files
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime

In [436]:
# Очистить журналы от предыдущих вызовов
!rm -rf ./logs/ 
!ls

CompaniesList.txt  forexQuotes.csv  sample_data  stockQuotes.csv


In [437]:
# Ввод ключа (IQLCQLXYBPW3LH7T)
key = '8C5RUT8LNE2P0F4O' #input('Введите ключ доступа   -   ') 

In [438]:
ts = TimeSeries(key = key)
# listCompany = ['MSFT', 'CAT', 'BA', 'GS', 'HD'] 

with open('CompaniesList.txt') as f:
  listTitleCompany = f.read()
  listCompany = listTitleCompany.split(" ")

In [439]:
# Скачивание исторических данных котировок акций
ts = TimeSeries(key = key, output_format = 'pandas')
# Счетчик для задержки между запросами из за количественного их ограничения сервером за минуту
pauseDownload = 0        
for i in range (len(listCompany)):
  # Обработка ошибок по API запросам
  try:
    # Задаем задержку по количеству запросов в минуту
    if pauseDownload % 30 == 0 and pauseDownload != 0:        
      time.sleep(61)
    if pauseDownload == 0:
      # Запрос данных через API
      data, meta_data = ts.get_intraday(symbol = listCompany[i], interval = '5min', outputsize = 'full')  
      # Преоброзование таблиц в нужный нам вид 
      data = data[['1. open', '5. volume']] 
      data.columns = [listCompany[i] + '-P',listCompany[i] + '-V']
      if pauseDownload % 10 == 0 and pauseDownload != 0:
        print(listCompany[i], data.shape[0], end = '\n')
      else:
        print(listCompany[i], data.shape[0], end = '   ')
    else:
      dataConcat, meta_data = ts.get_intraday(symbol = listCompany[i],interval = '15min', outputsize = 'full')
      dataConcat = dataConcat[['1. open', '5. volume']]
      dataConcat.columns = [listCompany[i] + '-P',listCompany[i] + '-V']
      if pauseDownload % 10 == 0 and pauseDownload != 0:
        print(listCompany[i], dataConcat.shape, end = '\n')
      else:
        print(listCompany[i], dataConcat.shape, end = '   ')
      # Объединение таблиц
      data = pd. concat([data, dataConcat], axis = 1)
  except ValueError:
    print(listCompany[i], "Что-то пошло не так")
  pauseDownload += 1
print(data)

MMM 1823   ABT (1143, 2)   ABBV (1243, 2)   ACN (882, 2)   ATVI (1368, 2)   ADBE (1178, 2)   ADT (1153, 2)   AAP (865, 2)   AET Что-то пошло не так
AMG (786, 2)   AFL (919, 2)
A (868, 2)   APD (839, 2)   ARG Что-то пошло не так
AKAM (945, 2)   AA (1224, 2)   ALXN (884, 2)   ALLE (818, 2)   AGN Что-то пошло не так
ADS (918, 2)   ALL (888, 2)
GOOG (1095, 2)   MO (1296, 2)   AMZN (1540, 2)   AEP (907, 2)   AEE (837, 2)   AXP (1165, 2)   AIG (1167, 2)   AMT (869, 2)   AMP (844, 2)   ABC (862, 2)
AME (823, 2)   AMGN (999, 2)   APH (825, 2)   APC Что-то пошло не так
ADI (965, 2)   AAL (1912, 2)   ANTM (849, 2)   AON (843, 2)   APA (1397, 2)   AIV (828, 2)
AAPL (1910, 2)   AMAT (1340, 2)   ADM (880, 2)   AIZ (827, 2)   T (1697, 2)   ADSK (928, 2)   ADP (922, 2)   AN (844, 2)   AZO (821, 2)   AVGO (1116, 2)
AVB (838, 2)   AVY (824, 2)   BHI Что-то пошло не так
BLL (862, 2)   BAC (1841, 2)   BXTL Что-то пошло не так
BAX (852, 2)   BBT Что-то пошло не так
BDX (899, 2)   BBBY (1213, 2)
BRK-B (131

In [440]:
# Скачивание исторических данных курсов валют
fe = ForeignExchange(key = key, output_format = 'pandas')
listF = ['EUR', 'BTC', 'CHF', 'JPY', 'CNY', 'GBP', 'RUB']
for i in listF:
  # Обработка ошибок по API запросам
  try:     
    if i == listF[0]:
      # Запрос данных через API
      dataF, meta_dataF = fe.get_currency_exchange_intraday(from_symbol = i, to_symbol = 'USD', outputsize = 'full') 
      # Преоброзование таблиц в нужный нам вид 
      dataF = pd.DataFrame(dataF['1. open'])
      dataF.columns = [i]
    else:
      dataConcatF, meta_dataF = fe.get_currency_exchange_intraday(from_symbol = i, to_symbol = 'USD', outputsize = 'full') 
      dataConcatF = pd.DataFrame(dataConcatF['1. open'])
      dataConcatF.columns = [i]
      dataF = pd. concat([dataF, dataConcatF], axis = 1)
    print(i, dataF.shape[0])
  except ValueError:
    print(i, "Что-то пошло не так", end = "\n")
print(dataF)

EUR 237
BTC Что-то пошло не так
CHF 237
JPY 237
CNY 818
GBP 818
RUB 818
                        EUR     CHF     JPY     CNY     GBP     RUB
date                                                               
2020-09-02 07:45:00     NaN     NaN     NaN  0.1465     NaN     NaN
2020-09-02 08:00:00     NaN     NaN     NaN  0.1465     NaN     NaN
2020-09-02 08:15:00     NaN     NaN     NaN  0.1465     NaN     NaN
2020-09-02 08:30:00     NaN     NaN     NaN  0.1465     NaN     NaN
2020-09-02 08:45:00     NaN     NaN     NaN  0.1465     NaN     NaN
...                     ...     ...     ...     ...     ...     ...
2020-09-14 18:30:00  1.1861  1.1014  0.0095     NaN  1.2858  0.0132
2020-09-14 18:45:00  1.1863  1.1015  0.0095     NaN  1.2857  0.0132
2020-09-14 19:00:00  1.1863  1.1014  0.0095     NaN  1.2854  0.0132
2020-09-14 19:15:00  1.1865  1.1012  0.0095     NaN  1.2855  0.0132
2020-09-14 19:30:00  1.1867  1.1012  0.0095     NaN  1.2861  0.0132

[818 rows x 6 columns]


In [441]:
allData = pd. concat([data, dataF], axis = 1)
allData.shape

(3537, 838)

In [442]:
# Определение времени для выбрки времени
dayStart = datetime.now() - timedelta(days = 4)
dayStart = dayStart.replace(hour=4, minute=0, second=0, microsecond=0)
dayEnd = datetime.now() - timedelta(days = 1)
dayEnd = dayEnd.replace(hour=20, minute=0, second=0, microsecond=0)
print(dayStart)
print(dayEnd)

2020-09-10 04:00:00
2020-09-13 20:00:00


In [443]:
# Изменяем формат даты в индексе и вырезаем по времени нужный отрезок
#allData.set_index(allData.index.strftime("%H:%M %d-%m-%Y"), inplace=True)
allDataSlice = allData.loc[dayStart:dayEnd]
allDataSlice.shape

(286, 838)

In [444]:
# Заполнение пропущенных значений
allDataSlice = allDataSlice.fillna(method='ffill')
allDataSlice = allDataSlice.fillna(method='bfill')

In [445]:
allDataSlice.iloc[250:255, 824:844]

,XYL-P,XYL-V,YUM-P,YUM-V,ZION-P,ZION-V,ZTS-P,ZTS-V,EUR,CHF,JPY,CNY,GBP,RUB
date,,,,,,,,,,,,,,
2020-09-11 15:20:00,84.445,24339.0,92.23,32630.0,31.33,32925.0,158.90,23070.0,1.1829,1.0995,0.0094,0.1463,1.2781,0.0133
2020-09-11 15:25:00,84.445,24339.0,92.23,32630.0,31.33,32925.0,158.90,23070.0,1.1829,1.0995,0.0094,0.1463,1.2781,0.0133
2020-09-11 15:30:00,84.520,37489.0,92.19,38879.0,31.32,26550.0,158.59,28739.0,1.1837,1.0998,0.0094,0.1463,1.2797,0.0133
2020-09-11 15:35:00,84.520,37489.0,92.19,38879.0,31.32,26550.0,158.59,28739.0,1.1837,1.0998,0.0094,0.1463,1.2797,0.0133
2020-09-11 15:40:00,84.520,37489.0,92.19,38879.0,31.32,26550.0,158.59,28739.0,1.1837,1.0998,0.0094,0.1463,1.2797,0.0133


In [446]:
allDataSliceTime = allDataSlice.query('index.dt.hour > 3 and index.dt.hour < 20')

In [447]:
allDataSliceTime.shape

(244, 838)

In [448]:
print(allDataSliceTime)

                      MMM-P  MMM-V  ABT-P  ...     CNY     GBP     RUB
date                                       ...                        
2020-09-10 04:00:00  164.96  185.0  106.2  ...  0.1462  1.3004  0.0132
2020-09-10 04:15:00  164.96  185.0  106.2  ...  0.1462  1.3004  0.0132
2020-09-10 04:30:00  164.96  185.0  106.2  ...  0.1462  1.3004  0.0132
2020-09-10 04:45:00  164.96  185.0  106.2  ...  0.1462  1.3004  0.0132
2020-09-10 05:00:00  164.96  185.0  106.2  ...  0.1462  1.3004  0.0132
...                     ...    ...    ...  ...     ...     ...     ...
2020-09-11 18:45:00  166.17  244.0  104.4  ...  0.1463  1.2788  0.0133
2020-09-11 19:00:00  166.17  244.0  104.4  ...  0.1463  1.2793  0.0133
2020-09-11 19:15:00  166.17  244.0  104.4  ...  0.1463  1.2791  0.0133
2020-09-11 19:30:00  166.17  244.0  104.4  ...  0.1463  1.2792  0.0133
2020-09-11 19:45:00  166.17  244.0  104.4  ...  0.1463  1.2799  0.0133

[244 rows x 838 columns]


In [449]:
# Сохранение и скачивание данных
allDataSliceTime.to_csv('stockQuotes.csv')
files.download('stockQuotes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>